In [14]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as vmodels
import torchaudio.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob
import soundfile as sf
from scipy.interpolate import interp1d
import IPython.display as ipd
import sys
import time
import pprint
from nnutils import reshape_mel_spec, RainforestNetV2, Params

In [15]:
# Hyper parameters
params = Params()
print (params.__to_dict__())

{'EPOCHS': 50, 'TRAIN_BATCH_SIZE': 4, 'LR': 0.0001, 'RAND_NOISE_UPPER': 0.1, 'RAND_NOISE_LOWER': 0.05, 'N_MELS': 224, 'SAMPLE_RATE': 48000, 'N_FFT': 5000, 'F_MIN': 90.0, 'F_MAX': 14000.0}


In [16]:
# CUDA
is_cuda = True and torch.cuda.is_available()
device = torch.device('cuda') if is_cuda else torch.device('cpu')

print ('cuda: ', is_cuda)

torch.cuda.empty_cache()

cuda:  True


In [17]:
test_files = glob.glob( 'data/test/*.flac' )

In [18]:
test_res = RainforestNetV2()
test_res.load_state_dict(torch.load('models/rainforest1608843907.662235.pt'))
test_res.eval()
print (test_res.training)
test_res = test_res.to(device)

False


In [6]:
#test_files.head()

In [7]:
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

In [8]:
results = pd.DataFrame()
print (device)

start_time = time.time()

mel_trans = transforms.MelSpectrogram(sample_rate=params.SAMPLE_RATE, n_mels=params.N_MELS, n_fft=params.N_FFT, f_min=params.F_MIN, f_max=params.F_MAX).to(device)

for f in tqdm(range(len(test_files))):
    #ex = test_files.iloc[idx]
    path = test_files[f]
    waveform, sample_rate = torchaudio.load(path)
    waveform = waveform.to(device)
    
    mel_spec = mel_trans(waveform)
    
    res = torch.zeros(24).to(device)
    with torch.no_grad():
        mel_spec = mel_spec.repeat(3, 1, 1)
        mel_spec = mel_spec.unsqueeze(0)
        mel_spec = mel_spec.to(device)

        output = test_res(mel_spec)[0]
        res = res+output
    
    res = res.to(torch.device('cpu')).numpy()    
    results = results.append({ 'result': res }, ignore_index=True)

end_time = time.time()
print ('\n-------- DONE --------')
print ('start time: {}'.format(start_time))
print ('end time: {}\n'.format(end_time))

duration = end_time-start_time
print ('testing duration: {}'.format(duration))

cuda




-------- DONE --------
start time: 1608844067.9482539
end time: 1608844213.5595324

testing duration: 145.61127853393555


In [9]:
np.set_printoptions(suppress=True, threshold=sys.maxsize)
results.head()

,result
0,"[0.078973874, 0.06721929, 0.078984186, 0.61286..."
1,"[0.05860769, 0.068104684, 0.062475156, 0.55882..."
2,"[0.085184984, 0.081337556, 0.085454985, 0.1360..."
3,"[0.9629003, 0.09849737, 0.07585755, 0.06292099..."
4,"[0.07171287, 0.07314431, 0.07082009, 0.0855429..."


In [10]:
submission = pd.DataFrame()

for idx in range(len(results)):
    ex = results.iloc[idx]
    recording_id = test_files[idx].replace('.flac', '').replace('data/test\\', '')
    result = ex.result
    
    species = 0
    o = {}
    for r in result:
        s = 's{}'.format(species)
        o[s] = r
        species += 1 
    o["recording_id"] = recording_id
    submission = submission.append(o, ignore_index=True)

In [11]:
cols = ['recording_id', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23']
submission = submission[cols]

In [12]:
submission.head()

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,0.078974,0.067219,0.078984,0.612862,0.069095,0.163462,0.238944,0.051775,0.077974,...,0.053447,0.089859,0.081821,0.123917,0.072684,0.114141,0.089841,0.074515,0.091755,0.087553
1,003bc2cb2,0.058608,0.068105,0.062475,0.558821,0.066234,0.064979,0.141251,0.086257,0.071711,...,0.047209,0.051390,0.366547,0.090765,0.076027,0.100275,0.084809,0.051803,0.074159,0.072912
2,0061c037e,0.085185,0.081338,0.085455,0.136067,0.085988,0.375832,0.352599,0.088961,0.081987,...,0.061683,0.095715,0.088121,0.131480,0.082336,0.168728,0.102916,0.096038,0.137964,0.141130
3,010eb14d3,0.962900,0.098497,0.075858,0.062921,0.068998,0.071244,0.100396,0.074007,0.075570,...,0.077534,0.079532,0.084595,0.079223,0.257452,0.079426,0.080237,0.073081,0.068508,0.078710
4,011318064,0.071713,0.073144,0.070820,0.085543,0.070030,0.060925,0.091752,0.066602,0.064507,...,0.954288,0.068642,0.084105,0.067214,0.065180,0.069332,0.064883,0.060741,0.082403,0.068262


In [13]:
np.set_printoptions(suppress=True, threshold=sys.maxsize)
submission.to_csv("submissions/submission-{}.csv".format(end_time), index=False, float_format='%.5f')